In [ ]:
import os
import numpy as np
from functions import *
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import fsolve
import scipy.signal as signal
import scipy.linalg as lina
import matplotlib.cm as cm
# Get the viridis colormap
# my_cmap = cm.get_cmap('viridis')


# Get the directory where the current script is located
script_directory = os.getcwd()

# Change the current working directory to the script's directory
os.chdir(script_directory)

# Now the CWD is the same as the script's directory
# print("New Current Working Directory:", os.getcwd())

default_pars = obtain_default_pars('pars_list.csv')
# display(default_pars)

pars_list = create_custom_pars_list('pars_list.csv')
par = default_pars
# par = pars_list[0] # between 0 - 3; 1 is good; 3 to compare
display(par)

path_maker = par['label']
path = f"CSV/{path_maker}.csv"

if not os.path.exists(path):
    guess = {
        'guess_range_real':[-20,5,5],
        'guess_range_imag':[0,50,25]
    }
    save_dataframe_to_csv(*find_eig(par, **guess, round_sig_digits=4, tol_is_sol=1e-6, max_iter=200), 'CSV')
# else:
    # print("Solution has already been saved in the appropriate location.")

df, label, metadata = plot_single_df(
        path, filter=True,
        real_lower_bound=-40, real_upper_bound=5, imag_lower_bound=-500, imag_upper_bound=500
)
n_lambdas = len(df)
# display(df.head(n_lambdas))

from scipy.integrate import quad

lambdas = []
for index, row in df.head(n_lambdas).iterrows():
    l = complex(row['Sol_r'], row['Sol_i'])
    # print(f'lambda_{index+1} = {l}')
    lambdas.append(l)

normal_coefs = []
for n in range(n_lambdas):
    l = lambdas[n]
    
    i_phi = quad(eig_fun_mul_0,0,1,args=(par, l),complex_func=True)[0]
    b_phi = np.sqrt(1/i_phi)
    
    i_psi = quad(eig_fun_mul_1,0,1,args=(par, l, [b_phi, 1]),complex_func=True)[0]
    b_psi = (1/i_psi)
    
    b = (b_phi, b_psi.conjugate())
    normal_coefs.append(b)
    
    # print(f'normal_coef_phi_{n+1} = {complex(round(normal_coefs[n][0].real,4),round(normal_coefs[n][0].imag,4))}')
    # print(f'normal_coef_psi_{n+1} = {complex(round(normal_coefs[n][1].real,4),round(normal_coefs[n][1].imag,4))}')
    # print()

In [2]:
from scipy.integrate import solve_ivp
import plotly.graph_objs as go

N_zeta = 100
zeta = np.linspace(0,1,N_zeta)
dz = zeta[1]
phi_1_0 = init_cond_func_1(zeta)
phi_2_0 = init_cond_func_2(zeta, par)
x0 = np.array([phi_1_0, phi_2_0])
np.random.seed(42)  # Set the seed for reproducibility
noise = np.random.normal(0, 0.1, size=x0.shape)
x_hat_0 = x0 + noise

In [3]:
t_end = 12
N_t_d = 5*t_end

n_modes=3

In [ ]:
t_eval_d = np.linspace(0,t_end,N_t_d+1)
s = 2/t_eval_d[1]
# G, S, S_u = G_S_Su(zeta, s, par, q_ctrl, N, r_ctrl, lambdas, n_modes, normal_coefs)
print(f't_eval_d = {t_eval_d[0]}, {t_eval_d[1]}, ... , {t_eval_d[-1]}')
# find_smallest_r(G, r_ctrl, 0, 5*r_ctrl)

In [ ]:
N = 9
q_ctrl = 1
r_ctrl = 675

u_limits = [-10,10]
# y_limits = [-0.2,0.2]

G, S, S_u = G_S_Su(zeta, s, par, q_ctrl, N, r_ctrl, lambdas, n_modes, normal_coefs)

import quadprog
x_MPC = []
x_hat_MPC = []
U_MPC = []
results_MPC = []
x_MPC.append(x0)
x_hat_MPC.append(x_hat_0)
y_MPC = []

L = np.ones(2*N_zeta) * 10
# L = np.linspace(1,0,2*N_zeta) * 2.5

L = L.reshape((2,N_zeta))

for i in range(len(t_eval_d)):
    G, a, C, b, meq = MPC_matrix_build(
            x_hat_MPC[i], s, par, 
            lambdas, normal_coefs, q_ctrl, G, S, S_u,
            n_modes=n_modes, N=N,
            u_limits=u_limits, y_limits=(-1000, 1000),
            meq=1
        )
    # display(G, a, C, b, meq)
    results_MPC.append(quadprog.solve_qp(G, a, C, b, meq))
    U_MPC.append(results_MPC[i][0][0])
    
    x_MPC.append(A_d(x_MPC[i], s, par)[:,:,-1] + B_d(zeta, s, par, u=U_MPC[i]))
    x_MPC[-1][1,-1] = x_MPC[-1][0,-1].copy()
    
    y_MPC.append(C_d(x_MPC[i], s, par) + D_d(zeta, s, par) * U_MPC[i])
    y_hat = C_d_obs(x_hat_MPC[i], s, par, L) + D_d_obs(L, s, par) * U_MPC[i] + M_d_obs(L, s, par) * y_MPC[i]
    
    
    t1 = A_d(x_hat_MPC[i], s, par)[:,:,-1]
    t2 = B_d(zeta, s, par, u=U_MPC[i])
    t3 = L_d(L, s, par) * (y_MPC[i] - y_hat)
    
    x_hat_MPC.append(t1 + t2 + t3)
    x_hat_MPC[-1][1,-1] = x_hat_MPC[-1][0,-1].copy()
    
    print(np.round(results_MPC[i][0],3))
    print(f"u(t) = {results_MPC[i][0][0]:.4f} obtained at t = {t_eval_d[i] + t_eval_d[1]:.2f} s")
    print(f"y(t) = {y_MPC[i]:.2f} and y_hat(t) = {y_hat:.2f} obtained at t = {t_eval_d[i] + t_eval_d[1]:.2f} s")

x_MPC = np.transpose(np.array(x_MPC[:-1]),(1,2,0))
x_hat_MPC = np.transpose(np.array(x_hat_MPC[:-1]),(1,2,0))

U_MPC = np.array([0] + U_MPC[:-1])

# import quadprog
# x_MPC = []
# U_MPC = []
# results_MPC = []
# x_MPC.append(x0)
# for i in range(len(t_eval_d)):
#     G, a, C, b, meq = MPC_matrix_build(
#             x_MPC[i], s, par, 
#             lambdas, normal_coefs, q_ctrl, G, S, S_u,
#             n_modes=n_modes, N=N,
#             u_limits=u_limits, y_limits=(-1000, 1000),
#             meq=1
#         )
#     # display(G, a, C, b, meq)
#     results_MPC.append(quadprog.solve_qp(G, a, C, b, meq))
#     U_MPC.append(results_MPC[i][0][0])
#     x_MPC.append(A_d(x_MPC[i], s, par)[:,:,-1] + B_d(zeta, s, par, u=U_MPC[i]))
#     x_MPC[-1][1,-1] = x_MPC[-1][0,-1].copy()
#     print(f"u(t) = {results_MPC[i][0][0]:.4f} obtained at t = {t_eval_d[i] + t_eval_d[1]:.2f} s")

# x_MPC = np.transpose(np.array(x_MPC[:-1]),(1,2,0))
# U_MPC = np.array([0] + U_MPC[:-1])


In [ ]:
# Set Times font for the plot
plt.rc('font', family='serif', serif='Times')
plt.rc('text', usetex=True)
t_eval_d *= 100
# Create t_grid and z_grid for meshgrid
t_grid, z_grid = np.meshgrid(t_eval_d, zeta)

# Create a figure for xd[0] and xd[1] wireframe plots
fig = plt.figure(figsize=(5,5))
azim_angle = 150
# Plot xd[0]
ax1 = fig.add_subplot(111, projection='3d')
ax1.view_init(azim=azim_angle)  # For xd[0] plot
ax1.plot_wireframe(z_grid, t_grid, x_MPC[0], color='black', linewidth=0.8, rstride=3, cstride=1)
ax1.set_ylabel(r'$t$, s')
ax1.set_xlabel(r'$\zeta$, m')
ax1.set_zlabel(r'$x_1(\zeta, t)$, M')
# ax1.set_title(r'Open-loop Response $x_1(\zeta, t)$')

# Plot xd[1]
# ax2 = fig.add_subplot(212, projection='3d')
# ax2.view_init(azim=azim_angle)  # For xd[1] plot
# ax2.plot_wireframe(z_grid, t_grid, x_MPC[1], color='black', linewidth=0.8, rstride=3, cstride=1)
# ax2.set_ylabel(r'$t$, s')
# ax2.set_xlabel(r'$\zeta$, m')
# ax2.set_zlabel(r'$x_2(\zeta, t)$, M')

# Adjust layout and display the plots
plt.tight_layout()
plt.savefig('closedloop_response.svg', format='svg')
plt.show()



In [ ]:
# Set Times font for the plot
plt.rc('font', family='serif', serif='Times')
plt.rc('text', usetex=True)
# t_eval_d *= 100
# Create t_grid and z_grid for meshgrid
t_grid, z_grid = np.meshgrid(t_eval_d, zeta)

# Create a figure for xd[0] and xd[1] wireframe plots
fig = plt.figure(figsize=(5,5))
azim_angle = 150
# Plot xd[0]
ax1 = fig.add_subplot(111, projection='3d')
ax1.view_init(azim=azim_angle)  # For xd[0] plot
ax1.plot_wireframe(z_grid, t_grid, x_hat_MPC[0], color='black', linewidth=0.8, rstride=3, cstride=1)
ax1.set_ylabel(r'$t$, s')
ax1.set_xlabel(r'$\zeta$, m')
ax1.set_zlabel(r'$x_1(\zeta, t)$, M')
# ax1.set_title(r'Open-loop Response $x_1(\zeta, t)$')

# Plot xd[1]
# ax2 = fig.add_subplot(212, projection='3d')
# ax2.view_init(azim=azim_angle)  # For xd[1] plot
# ax2.plot_wireframe(z_grid, t_grid, x_MPC[1], color='black', linewidth=0.8, rstride=3, cstride=1)
# ax2.set_ylabel(r'$t$, s')
# ax2.set_xlabel(r'$\zeta$, m')
# ax2.set_zlabel(r'$x_2(\zeta, t)$, M')

# Adjust layout and display the plots
plt.tight_layout()
plt.savefig('closedloop_response.svg', format='svg')
plt.show()



In [ ]:
# Set Times font for the plot
plt.rc('font', family='serif', serif='Times')
plt.rc('text', usetex=True)
# t_eval_d *= 100
# Create t_grid and z_grid for meshgrid
t_grid, z_grid = np.meshgrid(t_eval_d, zeta)

# Create a figure for xd[0] and xd[1] wireframe plots
fig = plt.figure(figsize=(5,5))
azim_angle = 150
# Plot xd[0]
ax1 = fig.add_subplot(111, projection='3d')
ax1.view_init(azim=azim_angle)  # For xd[0] plot
ax1.plot_wireframe(z_grid, t_grid, (x_MPC[0] - x_hat_MPC[0]), color='black', linewidth=0.8, rstride=3, cstride=1)
ax1.set_ylabel(r'$t$, s')
ax1.set_xlabel(r'$\zeta$, m')
ax1.set_zlabel(r'$x_1(\zeta, t)$, M')
# ax1.set_title(r'Open-loop Response $x_1(\zeta, t)$')

# Plot xd[1]
# ax2 = fig.add_subplot(212, projection='3d')
# ax2.view_init(azim=azim_angle)  # For xd[1] plot
# ax2.plot_wireframe(z_grid, t_grid, x_MPC[1], color='black', linewidth=0.8, rstride=3, cstride=1)
# ax2.set_ylabel(r'$t$, s')
# ax2.set_xlabel(r'$\zeta$, m')
# ax2.set_zlabel(r'$x_2(\zeta, t)$, M')

# Adjust layout and display the plots
plt.tight_layout()
plt.savefig('closedloop_response.svg', format='svg')
plt.show()



In [ ]:
plt.figure(figsize=(5, 3.5))
plt.step(t_eval_d, U_MPC, where='pre', color='gray', linewidth=2)  # Change to stairs plot
# plt.plot(t_eval_d, np.ones_like(t_eval_d)*u_limits[0], 'k--')
# plt.plot(t_eval_d, np.ones_like(t_eval_d)*u_limits[1], 'k--')
plt.grid(which='both', linestyle='--', linewidth=0.75)
plt.gca().xaxis.set_major_locator(plt.MaxNLocator(5))
plt.gca().yaxis.set_major_locator(plt.MaxNLocator(4))
# plt.title('Control input u(t) vs. time')
plt.gca().set_xlabel('Time (s)')
plt.gca().set_ylabel('Control input u(t)')
plt.legend(['u(t)', 'Input constraints'])
# plt.savefig('input.svg', format='svg')
plt.show()

In [ ]:
y_MPC = x_MPC[0,-1,:]
# Create a figure with the desired size
fig, ax1 = plt.figure(figsize=(4.5, 3.2)), plt.gca()

# Plot for u(t) on the primary y-axis (left)
ax1.step(t_eval_d, U_MPC, where='pre', color='black', linestyle='-', linewidth=2, label='u(t)')  # Solid black for u(t)
ax1.plot(t_eval_d, np.ones_like(t_eval_d) * u_limits[0], color='gray', linestyle='--', linewidth=1.5, label='Input constraints')  # Dashed gray for lower constraint
ax1.plot(t_eval_d, np.ones_like(t_eval_d) * u_limits[1], color='gray', linestyle='--', linewidth=1.5)  # Dashed gray for upper constraint

# Configure the primary y-axis (left)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Control input u(t), M')
ax1.grid(which='both', linestyle='--', linewidth=0.75, color='lightgray')  # Lighter gray gridlines
ax1.set_ylim([0, 0.25])
ax1.xaxis.set_major_locator(plt.MaxNLocator(5))
ax1.yaxis.set_major_locator(plt.MaxNLocator(5))

# Create a second y-axis (right) for y(t)
ax2 = ax1.twinx()  # Instantiate a second y-axis sharing the same x-axis

# Plot for y(t) on the secondary y-axis (right)
ax2.plot(t_eval_d, y_MPC, color='dimgray', linestyle='-', linewidth=1.5, label='y(t)')  # Dim gray for y(t)

ax2.set_ylim([0, 1])
# Configure the secondary y-axis (right)
ax2.set_ylabel(r'Reactor output $y(t)$, M', color='k')  # Keep labels black

# Combine legends from both ax1 and ax2
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()

# Add the combined legend
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper right', frameon=False)

# Save the figure (optional)
plt.savefig('input.svg', format='svg')

# Display the plot
plt.show()


In [ ]:
from scipy.interpolate import CubicSpline
y_MPC = x_MPC[0,-1,:]

# Create a finer time vector for interpolation (e.g., 4x more points)
t_fine = np.linspace(t_eval_d[0], t_eval_d[-1], len(t_eval_d) * 4)

# Perform cubic spline interpolation to get the smoother y_MPC
cs = CubicSpline(t_eval_d, y_MPC)  # Cubic spline interpolation
y_MPC_smooth = cs(t_fine)  # Get the interpolated values for the finer time grid

# Create a figure with the desired size
fig, ax1 = plt.figure(figsize=(4.5, 3.2)), plt.gca()

# Plot for u(t) on the primary y-axis (left)
ax1.step(t_eval_d, U_MPC, where='pre', color='black', linestyle='-', linewidth=2, label='u(t)')  # Solid black for u(t)
ax1.plot(t_eval_d, np.ones_like(t_eval_d) * u_limits[0], color='gray', linestyle='--', linewidth=1.5, label='Input constraints')  # Dashed gray for lower constraint
ax1.plot(t_eval_d, np.ones_like(t_eval_d) * u_limits[1], color='gray', linestyle='--', linewidth=1.5)  # Dashed gray for upper constraint

# Configure the primary y-axis (left)
ax1.set_xlabel('Time (s)')
ax1.set_ylabel('Control input u(t), M')
ax1.grid(which='both', linestyle='--', linewidth=0.75, color='lightgray')  # Lighter gray gridlines
ax1.set_ylim([0, 0.25])
ax1.xaxis.set_major_locator(plt.MaxNLocator(5))
ax1.yaxis.set_major_locator(plt.MaxNLocator(5))

# Create a second y-axis (right) for y(t)
ax2 = ax1.twinx()  # Instantiate a second y-axis sharing the same x-axis

# Plot for y(t) on the secondary y-axis (right)
ax2.plot(t_fine, y_MPC_smooth, color='dimgray', linestyle='-', linewidth=1.5, label='y(t)')  # Dim gray for y(t)

ax2.set_ylim([0, 1])
# Configure the secondary y-axis (right)
ax2.set_ylabel(r'Reactor output $y(t)$, M', color='k')  # Keep labels black

# Combine legends from both ax1 and ax2
lines_1, labels_1 = ax1.get_legend_handles_labels()
lines_2, labels_2 = ax2.get_legend_handles_labels()

# Add the combined legend
ax1.legend(lines_1 + lines_2, labels_1 + labels_2, loc='upper right', frameon=False)

# Save the figure (optional)
plt.savefig('input.svg', format='svg')

# Display the plot
plt.show()


In [10]:
x_openloop = A_d(x0, s, par, pow=len(t_eval_d))[:,:,:-1]


In [ ]:
# Set Times font for the plot
plt.rc('font', family='serif', serif='Times')
plt.rc('text', usetex=True)

# Create t_grid and z_grid for meshgrid
t_grid, z_grid = np.meshgrid(t_eval_d, zeta)

# Create a figure for xd[0] and xd[1] wireframe plots
fig = plt.figure(figsize=(5,5))
azim_angle = 150
# Plot xd[0]
ax1 = fig.add_subplot(111, projection='3d')
ax1.view_init(azim=azim_angle)  # For xd[0] plot
ax1.plot_wireframe(z_grid, t_grid, x_openloop[0], color='black', linewidth=0.8, rstride=3, cstride=1)
ax1.set_ylabel(r'$t$, s')
ax1.set_xlabel(r'$\zeta$, m')
ax1.set_zlabel(r'$x_1(\zeta, t)$, M')
# ax1.set_title(r'Open-loop Response $x_1(\zeta, t)$')

# Plot xd[1]
# ax2 = fig.add_subplot(212, projection='3d')
# ax2.view_init(azim=azim_angle)  # For xd[1] plot
# ax2.plot_wireframe(z_grid, t_grid, x_openloop[1], color='black', linewidth=0.8, rstride=3, cstride=1)
# ax2.set_ylabel(r'$t$, s')
# ax2.set_xlabel(r'$\zeta$, m')
# ax2.set_zlabel(r'$x_2(\zeta, t)$, M')

# Adjust layout and display the plots
plt.tight_layout()
plt.savefig('openloop_response.svg', format='svg')
plt.show()

## adding observer for ECC
- $C_d$, $D_d$ DONE
- $L_{cont}$ and $\lambda$ distribution
    - Constant $L(\zeta) = L$
    - Linear $L(\zeta) = L \times \zeta$
- $C_{d,o}, D_{d,o}, M_{d,o}, L_{d,o}$
- add observer dynamics in MPC receding horizon
- MPC QP


In [2]:
from scipy.sparse import csr_matrix, vstack, hstack
from scipy.integrate import solve_ivp
from scipy.sparse.linalg import eigs
import plotly.graph_objs as go

N_zeta = 200
zeta = np.linspace(0,1,N_zeta)
dz = zeta[1]

# Define the time points for the solution
t_end = 10
N_t = 50
t_eval = np.linspace(0, t_end, N_t+1)  # Time points at which to evaluate
dt = t_eval[1]

(k, v, D, tau, R) = (par['k'], par['v'], par['D'], par['tau'], par['R'])
# k = -1

# Lists to hold data for middle rows
rows_data = [
    k - (2*D)/(dz**2) - (v)/(dz) - (v**2)/(2*D),
    (2*D)/(dz**2),
    R * ((v)/(dz) + (v**2)/(2*D))]
rows_row_indices = [0,0,0]
rows_col_indices = [0,1,N_zeta]

# Fill the middle rows (from index 1 to N_zeta-2)
for i in range(1, N_zeta-1):
    # Fill the element before the diagonal (3), if it exists
    rows_data.extend([
        (D)/(dz**2) + (v)/(2*dz),
        k - (2*D)/(dz**2),
        (D)/(dz**2) - (v)/(2*dz)
    ])
    rows_row_indices.extend([i, i, i])
    rows_col_indices.extend([i-1, i, i+1])

# Define the last row
rows_data.extend([
    (2*D)/(dz**2),
    k - (2*D)/(dz**2)
])
rows_row_indices.extend([N_zeta-1, N_zeta-1])
rows_col_indices.extend([N_zeta-2, N_zeta-1])

for i in range(N_zeta, 2*N_zeta-2):
    rows_data.extend([
        -1/(tau*dz),
        1/(tau*dz)
        ])
    rows_row_indices.extend([i,i])
    rows_col_indices.extend([i,i+1])

# Define the last row
rows_data.extend([
        1/(tau*dz),
        -1/(tau*dz)
        ])
rows_row_indices.extend([2*N_zeta-2, 2*N_zeta-2])
rows_col_indices.extend([N_zeta-1, 2*N_zeta-2])


# Construct the middle rows as a sparse matrix
A_ode = csr_matrix((rows_data, (rows_row_indices, rows_col_indices)), shape=(2*N_zeta-1, 2*N_zeta-1))

In [75]:
row_indices = np.arange(2 * N_zeta - 1)
col_indices = np.full((2 * N_zeta - 1,), N_zeta - 1)
l_obs_range = np.linspace(0,10,81)
eigs_obs_L = []
eigs_obs_Lz = []

for l_obs in l_obs_range:
    
    L = np.ones(2*N_zeta-1) * l_obs
    Lz = np.linspace(0,1,2*N_zeta-1) * l_obs
    LC = csr_matrix((L, (row_indices, col_indices)), shape=(2 * N_zeta - 1, 2 * N_zeta - 1))
    LzC = csr_matrix((Lz, (row_indices, col_indices)), shape=(2 * N_zeta - 1, 2 * N_zeta - 1))
    A_obs_L = A_ode - LC
    A_obs_Lz = A_ode - LzC
    
    try:
        eig_obs, _ = eigs(A_obs_L, k=3, which='LR')
    except:
        eig_obs, _ = eigs(A_obs_L, k=15, which='LR')
    eig_obs = eig_obs[np.argsort(-eig_obs.real)]
    eigs_obs_L.append(eig_obs[0])
    
    try:
        eig_obs, _ = eigs(A_obs_Lz, k=3, which='LR')
    except:
        eig_obs, _ = eigs(A_obs_Lz, k=15, which='LR')
    eig_obs = eig_obs[np.argsort(-eig_obs.real)]
    eigs_obs_Lz.append(eig_obs[0])

In [ ]:
# Create the plot
plt.figure(figsize=(6, 4))

# Plot with smaller and more subtle markers
plt.plot(l_obs_range, np.real(eigs_obs_L), label='L', marker='^', color='black', linestyle='--', linewidth=0.8, markersize=3, markerfacecolor='grey', markeredgewidth=0.6)
plt.plot(l_obs_range, np.real(eigs_obs_Lz), label='Lz', marker='s', color='black', linestyle='-', linewidth=0.8, markersize=3, markerfacecolor='grey', markeredgewidth=0.6)

# Add a horizontal line at y=0
plt.axhline(y=0, color='black', linestyle='-', linewidth=1)

# Add labels and grid
plt.xlabel('Observation coefficient ($l_{obs}$)', fontsize=9)
plt.ylabel('Real Part of Eigenvalues', fontsize=9)
# plt.title('Real Part of Eigenvalues vs. Observation Range', fontsize=10)
plt.legend(fontsize=10, loc='upper right', frameon=False)

plt.grid(True, linestyle=':', linewidth=0.5)

# Adjust the layout for tight fit
plt.tight_layout()

# Display the plot
plt.show()